In [ ]:
#필요한 패키지 import

from tensorflow import keras
from tensorflow.keras.layers import Dense, Flatten

import tensorflow as tf
import IPython

#keras Tuner 설치

!pip install -U keras-tuner
import kerastuner as kt

In [ ]:
#데이터셋 불러오기

(X_train, y_train), (X_test, y_test) = keras.datasets.fashion_mnist.load_data()

#데이터 정규화

X_train = X_train.astype('float32') / 255.0
X_test = X_test.astype('float32') / 255.0

In [10]:
#모델 제작
def model_builder(hp):
    model = keras.Sequential()
    model.add(Flatten(input_shape=(28,28))) #입력층

    hp_units = hp.Int('units', min_value = 32, max_value = 512, step = 32)
    model.add(Dense(units= hp_units, activation = 'relu'))
    model.add(Dense(10, activation = 'softmax'))

    hp_learning_rate = hp.Choice('learning_rate', values = [1e-2, 1e-3, 1e-4])

    model.compile(optimizer = keras.optimizers.Adam(learning_rate = hp_learning_rate),
                  loss = keras.losses.SparseCategoricalCrossentropy(from_logits=True),
                  metrics = ['accuracy'])
    
    return model

#튜너 제작

tuner = kt.Hyperband(model_builder,
                     objective = 'val_accuracy',
                     max_epochs =  10,
                     factor = 3,
                     directory = 'my_dir',
                     project_name = 'intro_to_kt')

#callback 함수 지정

class ClearTrainingOutput(tf.keras.callbacks.Callback):
    def on_train_end(*args, **kwargs):
        IPython.display.clear_output(wait=True)

INFO:tensorflow:Reloading Oracle from existing project my_dir/intro_to_kt/oracle.json


In [11]:
#하이퍼파라미터 탐색

tuner.search(X_train, y_train, epochs = 10, validation_data = (X_test, y_test), callbacks = [ClearTrainingOutput()])


Trial 31 Complete [00h 00m 43s]
val_accuracy: 0.8499000072479248

Best val_accuracy So Far: 0.8881999850273132
Total elapsed time: 00h 15m 40s
INFO:tensorflow:Oracle triggered exit


In [12]:
#모델 평가

best_hps = tuner.get_best_hyperparameters(num_trials=1)[0]

print(f"""
하이퍼 파라미터 검색이 완료되었습니다. 
최적화된 첫 번째 Dense 노드 수는 {best_hps.get('units')} 입니다.
최적의 학습 속도는 {best_hps.get('learning_rate')} 입니다.
""")


하이퍼 파라미터 검색이 완료되었습니다. 
최적화된 첫 번째 Dense 노드 수는 320 입니다.
최적의 학습 속도는 0.001 입니다.

